In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import re
import time
import csv
import pickle
from pprint import pprint

In [208]:
#Proxies need to be changed each run until data is collected!
proxies = {
    "http": 'http://152.157.119.253:3128', 
    "https": 'http://152.157.119.253:3128'
}
prox2 = {
    "http": 'http://54.209.135.103:3128', 
    "https": 'http://54.209.135.103:3128'
}
prox3 = {
    "http": 'http://18.191.95.240:8080', 
    "https": 'http://18.191.95.240:8080'
}
prox4 = {
    "http": 'http://159.65.47.178:8888', 
    "https": 'http://159.65.47.178:8888'
}
superbase = 'https://steepster.com/'
start_url = 'https://steepster.com/teas?green=on'
search_base = 'https://steepster.com/teas?green=on&page=' #Add page number to end
ttype_list = ['Green Tea','Black Tea','Pu-Erh Tea','Oolong Tea','White Tea','Matcha Tea','Yellow Tea']

# Scraping Top Teas

Using BeautifulSoup to scrape te top teas.  Pulling the items from the page, and cleaning them. Results are stored in nested dictionarys.

## Scraping Section

Scraping the first 3 pages of the following tea types: Green, Matcha, Black, White, Yellow, Pu-erh, and Oolong.

In [249]:
def getTeas(color):
    review_url = []
    descpage = []
    proxy=None
    base = 'https://steepster.com/teas?'
    base = base+color
    for j in range(6):
        response = rq.get(base+str(j+1),proxy)
        page = response.text
        mainpage = soup(page,"lxml")
        #Getting all cases where we have a /tea url
        list_1 = mainpage.find_all(class_='entry-count')
        time.sleep(.5)
        for i in list_1:
            ret = re.findall("/teas/[a-zA-Z\-/0-9]+",str(i))
            descpage.append(superbase+ret[0])
            time.sleep(.5)
        if j ==3:
            proxy=proxies
    return descpage

In [163]:
len(descpage)

168

In [74]:
response = rq.get('https://steepster.com/teas/bird-pick-tea-and-herb/5862-premium-silky-green-tea')

In [75]:
page = response.text
reviewp = soup(page,"lxml")

In [267]:
def getReviews(reviewpage, currenttea, proxy):
    reviewstr=[]
    review_list=[]
    if str(reviewpage.find_all('div', class_='text')):
        kj = reviewpage.find_all('a')[-14:-12]
        reviewstr = str(reviewpage.find_all('div', class_='text')).replace('</p>\n</span>\n</div>, <div class="text">\n','')\
                .replace('\n<p>','').replace('<br/>\n',' ').replace('<span class="caps">','')\
                .replace('</span>','').replace('</p>','').replace(']','').replace('\n\n</div>','').split('<span itemprop="description">')
        review_list += reviewstr[1:]      
        if kj[0].get('href') == '?page=2#tasting-notes':
            next_1 = kj[0].get('href')
            response = rq.get(currenttea+next_1, proxy)
            page = response.text
            reviewp2 = soup(page,"lxml")
            reviewstr = str(reviewp2.find_all('div', class_='text')).replace('</p>\n</span>\n</div>, <div class="text">\n','')\
                .replace('\n<p>','').replace('<br/>\n',' ').replace('<span class="caps">','')\
                .replace('</span>','').replace('</p>','').replace(']','').replace('\n\n</div>','').split('<span itemprop="description">')
            review_list += reviewstr[1:] 
        time.sleep(.05)
        if kj[1].get('href') == '?page=3#tasting-notes':
            next_ = kj[1].get('href')
            response = rq.get(currenttea+next_, proxy)
            page = response.text
            reviewp3 = soup(page,"lxml")
            reviewstr = str(reviewp3.find_all('div', class_='text')).replace('</p>\n</span>\n</div>, <div class="text">\n','')\
                .replace('\n<p>','').replace('<br/>\n',' ').replace('<span class="caps">','')\
                .replace('</span>','').replace('</p>','').replace(']','').replace('\n\n</div>','').split('<span itemprop="description">')
            review_list += reviewstr[1:]  
        time.sleep(.05)
    return review_list


In [239]:
def updateDict(url_list, old_dict, teatype):
    dict_list =[]
    count=0
    proxy = proxies
    for i in url_list:
        time.sleep(.5)
        new_dict = {}
        teaname = ''
        teaing = ''
        teaflav = ''
        tot_reviews = []
        response = rq.get(i, proxy)
        page = response.text
        reviewp = soup(page,"lxml")
        teaname = str(reviewp.h1).split('"')[3]
        teaing = str(reviewp.find_all('dd')[1]).split('>')[1].replace('</dd','')
        teaflav = str(reviewp.find_all('dd')[2]).replace('<dd>', '').replace('</dd>', '')
        tot_reviews = getReviews(reviewp,i,proxy)
        new_dict['Tea Name'] =teaname
        new_dict['Tea Ingredients'] =teaing
        new_dict['Tea Flavors'] =teaflav
        new_dict['Tea Reviews'] =tot_reviews
        dict_list.append(new_dict)
        count+=1
        if count==8:
            proxy=prox2
        elif count==16:
            proxy=prox3
        elif count==24:
            proxy=prox4
        elif count==32:
            proxy=proxies
            count=0
    old_dict[teatype]=dict_list
    return old_dict

In [252]:
comurl = ['green=on&page=','black=on&page=','pu-erh=on&page=','oolong=on&page=','white=on&page=','matcha=on&page=','yellow=on&page=']
tot_dict = {}
for index, i in enumerate(comurl):
    descpage = getTeas(i)
    tot_dict = updateDict(descpage, tot_dict, ttype_list[index])

In [254]:
descpage

['https://steepster.com//teas/rishi-tea/5683-ancient-yellow-buds',
 'https://steepster.com//teas/teavana/43360-golden-dragon-yellow-tea',
 'https://steepster.com//teas/mark-t-wendell/23148-huo-shan-yellow-sprouting',
 'https://steepster.com//teas/the-jasmine-pearl-tea-merchants/40313-yellow-mu-dan',
 'https://steepster.com//teas/davidstea/52871-yellow-gem',
 'https://steepster.com//teas/ten-ren/30347-sun-moon-lake-high-mountain-oolong-tea',
 'https://steepster.com//teas/mandala-tea/27557-yellow-buds-huo-shan-huang-ya',
 'https://steepster.com//teas/august-uncommon-tea/56208-the-settlement',
 'https://steepster.com//teas/what-cha/48466-huoshan-huang-ya-yellow',
 'https://steepster.com//teas/harney-and-sons/21228-hunan-yellow-sprout-2012',
 'https://steepster.com//teas/life-in-teacup/11912-meng-ding-snow-bud-xue-ya',
 'https://steepster.com//teas/tribute-tea-company/42429-huo-mountain-yellow-buds-yellow-tea',
 'https://steepster.com//teas/canton-tea-co/2985-meng-ding-huang-ya',
 'https:/

In [256]:
for i in tot_dict:
    print(len(tot_dict[i]))

168
168
168
168
168
168
168


In [257]:
#Writing a pickle file to save data

with open('tea_data.pkl', 'wb') as picklefile:
    pickle.dump(tot_dict, picklefile)

In [136]:
if str(reviewp.find_all('div', class_='text')):
    reviewstr = str(reviewp.find_all('div', class_='text')).replace('</p>\n</span>\n</div>, <div class="text">\n','')\
                .replace('\n<p>','').replace('<br/>\n',' ').replace('<span class="caps">','')\
                .replace('</span>','').replace('</p>','').replace(']','').replace('\n\n</div>','').split('<span itemprop="description">')
reviewstr = reviewstr[1:] 

In [149]:
turnpg = []
specurl = currentea[0]
kj = reviewp.find_all('a')[-14:-12]
if kf[0] == '?page=2#tasting-notes':
    next_ = kf[0].get('href')
    turnpg.append(specurl+next_)
    turnpg
if kf[1] == '?page=3#tasting-notes':
    next_ = kf[0].get('href')
    turnpg.append(specurl+next_)
    turnpg

['?page=2#tasting-notes', '?page=3#tasting-notes']

## Scraping by username

The amount of data will be the 50 most logged teas (of all users) that the user has reviewed.

In [209]:
#establishing a starting point
starturl = 'https://steepster.com/Mercuryhime/teas'
baseurl2 = 'https://steepster.com'
popsort = '&sort=popular'
pagesort = '?page='
following = 'following'
teas = '/teas'
#overall url build is baseurl2 + username +pagesort+ number+ popsort where the number is replaced by the page number

In [276]:
response = rq.get(starturl)
page = response.text
reviewp = soup(page,"lxml")

In [87]:
#pulling out string 3 for url, and string 4 for tea name
teanamehtml = reviewp.find_all(class_='tea-name')
teanameurl=[]
teaname = []
for i in teanamehtml:
    qq=re.findall(r"([^\"\']*?[\/teas][^\"]*)",str(i))
    teanameurl.append(str(qq[3]))
    gg = re.findall(r'([^\"><n]*?[a-z][^<\"]*)', str(qq[4]))
    teaname.append(gg[0].replace('\n','').strip())

In [88]:
teaname

['Laoshan Black',
 'Fengqing Dragon Pearl Black Tea',
 'Mi Lan Dan Cong',
 'Momoko',
 'Vanilla Comoro',
 'Florence',
 'Strawberry Seduction',
 'Green and Fruity',
 'Cantaloupe Bai Mu Dan',
 'Plantation Mint']

In [89]:
teanameurl

['/teas/verdant-tea/20064-laoshan-black',
 '/teas/teavivre/26683-fengqing-dragon-pearl-black-tea',
 '/teas/canton-tea-co/3054-mi-lan-dan-cong',
 '/teas/lupicia/17769-momoko',
 '/teas/harney-and-sons/3589-vanilla-comoro',
 '/teas/harney-and-sons/3515-florence',
 '/teas/element-tea/13861-strawberry-seduction',
 '/teas/davidstea/20184-green-and-fruity',
 '/teas/52teas/17100-cantaloupe-bai-mu-dan',
 '/teas/bigelow/727-plantation-mint']

In [273]:
response = rq.get('https://steepster.com/Mercuryhime/'+following)
page = response.text
reviewp = soup(page,"lxml")

In [274]:
kool = reviewp.find_all(class_='details')
usernames =[]
for i in kool:
    qq=re.findall(r"([^\"\']*?[\/][^\"]*)",str(i))
    usernames.append(qq[0])

In [105]:
for i in range(400):
    time.sleep(.5)
    response = rq.get('https://steepster.com/teaequalsbliss/'+following+pagesort+str(i),proxies)
    page = response.text
    reviewp = soup(page,"lxml")
    kool = reviewp.find_all(class_='details')
    for i in kool:
        qq=re.findall(r"([^\"\']*?[\/][^\"]*)",str(i))
        usernames.append(qq[0])
usernames.append('/teaequalsbliss')

In [280]:
#400 users, depth of 7 tea pages
def superTealist(users):
    teanameurl=[]
    newurls = []
    teaname = []
    score=[]
    userdict = {}
    count=0
    proxy=proxies
    for i in users:
        time.sleep(.5)
        teaname = []
        scor=[]
        for j in range(6):
            time.sleep(.5)
            response = rq.get('https://steepster.com'+str(i)+teas+pagesort+str(j+1)+popsort,proxy)
            page = response.text
            reviewp = soup(page,"lxml")
            #pulling out string 3 for url, and string 4 for tea name
            newurls = reviewp.find_all(class_='tea-name')
            scores = reviewp.find_all(class_='rating awesome')
            for q in scores:
                score = re.sub("<div class=\"rating awesome\">",' ',str(q))
                score = re.sub("</div>",' ',str(score)).strip()
                scor.append(score)
            for k in newurls:
                qq=re.findall(r"([^\"\']*?[\/teas][^\"]*)",str(k))
                teanameurl.append(str(qq[3]))
                gg = re.findall(r'([^\"><n]*?[a-z][^<\"]*)', str(qq[4]))
                teaname.append(gg[0].replace('\n','').strip())
        userdict[i] = [teanameurl, teaname, scor]
        if count == 10:
            proxy=prox2
        elif count == 20:
            proxy=prox3
        elif count == 30:
            proxy=prox4
        elif count ==40:
            proxy=proxies
            count=0
        count+=1
    return teanameurl, teaname, userdict

In [281]:
teanameurl, teaname, userdict1 = superTealist(usernames)

In [282]:
#Writing a pickle file to save data

with open('user_data.pkl', 'wb') as picklefile:
    pickle.dump(userdict1, picklefile)

In [134]:
newurls = reviewp.find_all(class_='tea-name')
for i in newurls:
                qq=re.findall(r"([^\"\']*?[\/teas][^\"]*)",str(i))
                teanameurl.append(str(qq[3]))
                gg = re.findall(r'([^\"><n]*?[a-z][^<\"]*)', str(qq[4]))
                teaname.append(gg[0].replace('\n','').strip())

In [268]:
def updateDict2(url_list):
    dict_list =[]
    count=0
    proxy = proxies
    for index,i in enumerate(url_list):
        time.sleep(.1)
        new_dict = {}
        teaname = ''
        teabrand=''
        teaing = ''
        teaflav = ''
        tot_reviews = []
        if index not in [455,491,1133,2192,2304,3167]:
            response = rq.get(baseurl2+i, proxy)
            page = response.text
            reviewp = soup(page,"lxml")
            teaname = str(reviewp.h1).split('"')[3]
            teatype = str(re.findall(r"([^\"><n]*?[a-z][^<\"]*)",str(reviewp.find_all('dd')[0]))[3])
            teabrand= str(re.findall(r"([^\"><n]*?[A-Za-z][^<\"]*)",str(reviewp.find_all(itemprop= 'name')[1]))[2])
            tot_reviews = getReviews(reviewp,baseurl2+i,proxy)
            new_dict['Tea Name'] =teaname
            new_dict['Tea Brand'] = teabrand
            new_dict['Tea Type'] =teatype
            try:
                teaing = str(reviewp.find_all('dd')[1]).split('>')[1].replace('</dd','')
            except:
                print(i)
                print(index)
            try:
                teaflav = str(reviewp.find_all('dd')[2]).replace('<dd>', '').replace('</dd>', '')
            except:
                print(index)
            
            new_dict['Tea Ingredients'] =teaing
            new_dict['Tea Flavors'] =teaflav
            new_dict['Tea Reviews'] =tot_reviews
            dict_list.append(new_dict)
            count+=1
            if count==8:
                proxy=prox2
            elif count==16:
                proxy=prox3
            elif count==24:
                proxy=prox4
            elif count==32:
                proxy=proxies
                count=0
    return dict_list

In [270]:
myset = set(teanameurl)
teadictlist = updateDict2(myset)
    

In [ ]:
#removing the reviews not related to drinkable tea
"""removelist = [teadictlist[455],teadictlist[491],teadictlist[1133],teadictlist[2192],teadictlist[2304],teadictlist[3167]]
newtealist = [x for x in teadictlist if x not in removelist]"""

In [283]:
#Writing a pickle file to save data

with open('items_data.pkl', 'wb') as picklefile:
    pickle.dump(teadictlist, picklefile)